# Trabajo Práctico 2: Entrenamiento y evaluación de modelos #

## 1. Métrica ##

Las métricas que vamos a utilizar son **F-SCORE** y además **Re-Call**.

### Por qué F-SCORE? ###
La misma consiste en combinar precission y recall. Los valores estan cerca de 1, cuando precissión y recall son buenos, en cambio, F-SCORE comienza a caer si uno de los dos cae.

### Por qué Re-Call? ###
Re-Call nos permite medir que no se nos escapen interesados a las tarjetas de crédito, y en caso que NO esté interesado, el rechazo de la tarjeta, no afecta al negocio. En cambio si se escapa un caso de un cliente con interés, la empresa perdería la posibilidad de generar ingresos.

## Librerías ##


In [ ]:
%matplotlib inline
import warnings
import numpy as np
import pandas as pd 
import matplotlib
import keras
import h5py
import PIL
import seaborn as sns
import sklearn
import pytz
import plotly.graph_objects as go

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

# One Hot Encoder
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, QuantileTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn import datasets
import plotly.express as px 
pd.set_option('display.float_format',lambda x:'%.0f'% x) #Sacar notación científica en pandas
pd.options.display.max_columns = 0

plt.rcParams.update({
    "font.family": ["serif"],
    "font.sans-serif": ["Roboto"],
    "font.size": 9,
    "axes.labelsize": 11,
    "axes.titlesize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    'figure.figsize': (15.0, 4.0),
    'axes.grid': False,
    'axes.spines.left': True,
    'axes.spines.right': True,
    'axes.spines.top': True,
    'axes.spines.bottom': True,
})

np.set_printoptions(suppress=True)

warnings.filterwarnings('ignore')

## Train ##


In [ ]:
data_TC = pd.read_csv('train.csv')

BETTER_COLUMN_NAMES = {
    'ID': 'id',
    'Gender': 'sexo',
    'Age': 'edad',
    'Region_Code': 'codigo_region',
    'Occupation': 'ocupacion',
    'Channel_Code': 'codigo_canal',
    'Vintage': 'antiguedad',
    'Credit_Product': 'tiene_producto_credito_activo',
    'Avg_Account_Balance': 'saldo_promedio_cuenta',
    'Is_Active': 'es_activo',
    'Is_Lead': 'esta_interesado',
}
data_TC.rename(columns=BETTER_COLUMN_NAMES, inplace=True)

data_TC.set_index('id', inplace=True)


## Test ##


In [ ]:
data_Test = pd.read_csv('test.csv')

BETTER_COLUMN_NAMES2 = {
    'ID': 'id',
    'Gender': 'sexo',
    'Age': 'edad',
    'Region_Code': 'codigo_region',
    'Occupation': 'ocupacion',
    'Channel_Code': 'codigo_canal',
    'Vintage': 'antiguedad',
    'Credit_Product': 'tiene_producto_credito_activo',
    'Avg_Account_Balance': 'saldo_promedio_cuenta',
    'Is_Active': 'es_activo',
    'Is_Lead': 'esta_interesado',
}
data_Test.rename(columns=BETTER_COLUMN_NAMES2, inplace=True)

data_Test.set_index('id', inplace=True)

In [ ]:
data_Test.dtypes


## 2. Técnica feature engineering ##

- **Técnica Quantile Transformation(edades)** Utilizaremos esta técnica para llevar los datos a una distribución uniforme o normal, generando robustez para los outliers.

In [ ]:
number_of_quantile = 5
data_TC['quantile'],edges = pd.qcut(x=data_TC['edad'], q=number_of_quantile, labels=False, retbins=True )
print('values of edges: ' + str(edges))

#Evitar Nans:
ed = np.delete(edges, 0)
ed = np.delete(ed, -1)
ed = np.append(np.append([data_TC.edad.min()], [ed]), [data_TC.edad.max()])
print('\nvalues of ed: ' + str(ed))



# Transform the array as an IntervalIndex
Interval_Index = pd.IntervalIndex.from_breaks(ed,closed='right',dtype='interval[int64]')
print('\nvalues of Interval_Index: ' + str(Interval_Index))

# Create a column displaying the interval
data_TC['quantile_interval'] = pd.cut(data_TC['edad'], bins=Interval_Index)

dict_inter_quantile = pd.Series(data_TC['quantile_interval'].unique().sort_values(ascending=False), name='interval').reset_index()
dict_inter_quantile.columns = ['quantile', 'quantile_interval']
dict_inter_quantile = dict_inter_quantile.set_index('quantile_interval')

data_TC.sample(8)

In [ ]:

for col in data_TC.columns: 
    print(col) 

In [ ]:
df = px.data.tips()

fig = px.histogram(data_TC, x="edad",title='Distribución sin quantiles')
fig.update_layout(bargap=0.2)
fig.show()

fig = px.histogram(data_TC,x="quantile",title='Distribución por quantiles')
fig.update_layout(bargap=0.2)
fig.show()